In [1]:
import os

os.environ["WANDB_MODE"]="disabled"
os.environ["CUDA_VISIBLE_DEVICES"] = ""
#!export CUDA_VISIBLE_DEVICES=""

In [2]:
import argparse
import os
import pandas as pd
import numpy as np
from transformers import BertTokenizer,BertForSequenceClassification,Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset
from sklearn import preprocessing
import torch.nn as nn
from sklearn.utils import compute_class_weight
from sklearn.metrics import f1_score, classification_report
import pickle



In [3]:
le = preprocessing.LabelEncoder()
# use this to map categories to integers.


In [4]:
# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.


functions and class definitions


In [5]:
class MulticlassDataset(Dataset):

    def __init__(self, encodings, labels):
      self.encodings = encodings
      self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)
        

In [6]:
def preprocess(dataframe, selected_material_type=None):
  #convert the dataframe labels accordingly by the material type
  if selected_material_type!="None":
    new_df = dataframe.copy()
    for _, row in new_df.iterrows():
      if row['iSampleMaterial'].split("_")[0] == selected_material_type:
        continue #leave the label
      else:
        row['iSampleMaterial']="None" #set none as label
  else:
    new_df = dataframe.copy()   #flattened label version 
  new_df
  #convert labels into integers
  le.fit(new_df.iSampleMaterial)

  new_df['iSampleMaterial'] = le.transform(new_df.iSampleMaterial)
  print(" number of labels: ", len(le.classes_))
  #split data to training df, val df, test df
  sample_size = 10000
  fraction=sample_size/len(new_df)  # get about 500 samples
  sel_len = sample_size
  train_df, dev_df, test_df =  np.split(new_df.sample(frac=fraction, random_state=42),[int(.6*sel_len), int(.8*sel_len)])

  train_df.to_csv('output/train_df.csv')
  dev_df.to_csv('output/dev_df.csv')
  test_df.to_csv('output/test_df.csv')
 
  return train_df, dev_df, test_df


def create_dataset(dataframe, tokenizer):
  MAX_LENGTH = 80
  inputs = {
          "input_ids":[],
          "attention_mask":[]
        }
    
  features_columns =[x for x in dataframe.columns.values if x != 'iSampleMaterial']
  def create_concatenated_text(dataframe):
    """combine the columns text to create a single sentence"""
    sents= [] #text that is a concatenation of all columns
    for _, row in dataframe.iterrows():
      combined = ""
#      for col in features_columns:
      row_value = row["text"]
      if row_value!="" and type(row_value)==str:
          combined += row_value   # +" , "
      sents.append(combined)
    return sents

  sents = create_concatenated_text(dataframe)
  for sent in sents:
    tokenized_input = tokenizer(sent,max_length=MAX_LENGTH, padding='max_length', truncation=True)
    inputs["input_ids"].append(torch.tensor(tokenized_input["input_ids"]))
    inputs["attention_mask"].append(torch.tensor(tokenized_input["attention_mask"]))
 
  print("torch tensor dataframe columns:", dataframe.columns.values)
  #print("dataframe['iSampleMaterial']: ",dataframe['iSampleMaterial'].values )
  labels = torch.tensor(dataframe['iSampleMaterial'].values.tolist())
    
  return MulticlassDataset(inputs,labels)

def get_class_weights(dataframe):
  """computes the class weight and returns a list to account for class imbalance """
  labels = torch.tensor(dataframe['iSampleMaterial'].values.tolist())
  class_weights=compute_class_weight( class_weight ='balanced',classes = np.unique(labels),y = labels.numpy())

  total_class_weights =torch.tensor(class_weights,dtype=torch.float).to(device)
  return total_class_weights

def create_custom_trainer(class_weights):
  """creates custom trainer that accounts for class imbalance"""
  class CustomTrainer(Trainer):
      def compute_loss(self, model, inputs, return_outputs=False):
          labels = inputs.get("labels")
          # forward pass
          outputs = model(**inputs)
          logits = outputs.get("logits")
          # compute custom loss 
          loss_fct = nn.CrossEntropyLoss(weight=class_weights)
          loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
          return (loss, outputs) if return_outputs else loss
  return CustomTrainer





def train(selected_type, dataframe, tokenizer, batch_size, learning_rate, epochs,train_mode, output_dir):

  train_df, dev_df, test_df = preprocess(dataframe,selected_type)
  train_dataset = create_dataset(train_df, tokenizer)
  dev_dataset = create_dataset(dev_df,tokenizer)
  test_dataset = create_dataset(test_df,tokenizer)

  #load model
  model = BertForSequenceClassification.from_pretrained("allenai/scibert_scivocab_uncased", num_labels = len(le.classes_), )

  # Tell pytorch to run this model on the GPU.
  #desc = model.cuda()
  desc = model.to(device)

  training_args = TrainingArguments(
          output_dir= output_dir,     # output directory
          num_train_epochs=epochs,              # total number of training epochs
          per_device_train_batch_size=batch_size,  # batch size per device during training
          per_device_eval_batch_size=batch_size,   # batch size for evaluation
          learning_rate = learning_rate,
          warmup_steps=500,                # number of warmup steps for learning rate scheduler
          weight_decay=0.01, 
          load_best_model_at_end=True,            
          logging_dir=output_dir,            # directory for storing logs
          logging_steps=10,
          evaluation_strategy = "epoch", #To calculate metrics per epoch
          save_strategy = "epoch"
  )
  #get class weight
  class_weights = get_class_weights(train_df)
  CustomTrainer = create_custom_trainer(class_weights)

  if train_mode == "custom":
    trainer = CustomTrainer(model = model, args =training_args, train_dataset=train_dataset, eval_dataset=dev_dataset)
  else:
    trainer = Trainer(model = model, args =training_args, train_dataset=train_dataset, eval_dataset=dev_dataset)
  trainer.train()

In [7]:
def preprocess_2(dataframe):
    
  le.fit(dataframe.iSampleMaterial)
  print(" number of labels: ", len(le.classes_))
  print("label encoder classes:",le.classes_)
  rockint = le.transform(["mat:rock"])[0]
  #convert the dataframe labels accordingly by the material type
  # want to sample the different material types according to their frequency
  # generate separate dataframes for mineral, rock, sediment, rockorsediment, and soil

  # fraction of total for each class
  #min_n = .255
  #rock_n = .435
  #sed_n = .125
  #rocksed_n = .065
  #soil_n = .12

#try different distribution
  min_n = .205
  rock_n = .235
  sed_n = .225
  rocksed_n = .215
  soil_n = .12
  # total sample 
  sample_size = 1000
  #rand_state = int(42)
  rand_state = int(19)
  
  
  #split data to training df, val df, test df
  min_df = dataframe[dataframe["iSampleMaterial"]=="mat:mineral"].copy()   #flattened label version 
    # build the data frames
  print("min_df rowcount: ", len(min_df.index))
  this_n = int(round(min_n * sample_size, 0))  # weights='weight', axis=0,
  train_df_min, dev_df_min, test_df_min =  np.split(min_df.sample(n=this_n,   random_state=rand_state),[int(.6*this_n), int(.8*this_n)])
  print("finished min dataframe. this_n:", this_n, " split at:",int(.6*this_n), int(.8*this_n))

  rock_df = dataframe[dataframe["iSampleMaterial"]=="mat:rock"].copy()   #flattened label version 
    # build the data frames
  this_n = int(round(rock_n * sample_size, 0))  # weights='weight', axis=0,
  train_df_rock, dev_df_rock, test_df_rock =  np.split(rock_df.sample(n=this_n,   random_state=rand_state),[int(.6*this_n), int(.8*this_n)])
  print("finished rock dataframe. this_n:", this_n, " split at:",int(.6*this_n), int(.8*this_n))

    
  sed_df = dataframe[dataframe["iSampleMaterial"]=="mat:sediment"].copy()   #flattened label version 
    # build the data frames
  this_n = int(round(sed_n * sample_size, 0))  # weights='weight', axis=0,
  train_df_sed, dev_df_sed, test_df_sed =  np.split(sed_df.sample(n=this_n,   random_state=rand_state),[int(.6*this_n), int(.8*this_n)])
  print("finished sed dataframe. this_n:", this_n, " split at:",int(.6*this_n), int(.8*this_n))


  rocksed_df = dataframe[dataframe["iSampleMaterial"]=="mat:rockorsediment"].copy()   #flattened label version 
    # build the data frames
  this_n = int(round(rocksed_n * sample_size, 0)) # weights='weight', axis=0,
  train_df_rocksed, dev_df_rocksed, test_df_rocksed =  np.split(rocksed_df.sample(n=this_n,   random_state=rand_state),[int(.6*this_n), int(.8*this_n)])
  print("finished rocksed dataframe. this_n:", this_n, " split at:",int(.6*this_n), int(.8*this_n))


  soil_df = dataframe[dataframe["iSampleMaterial"]=="mat:soil"].copy()   #flattened label version 
    # build the data frames
  this_n = int(round(soil_n * sample_size, 0)) # weights='weight', axis=0,
  train_df_soil, dev_df_soil, test_df_soil =  np.split(soil_df.sample(n=this_n,   random_state=rand_state),[int(.6*this_n), int(.8*this_n)])
  print("finished soil dataframe. this_n:", this_n, " split at:",int(.6*this_n), int(.8*this_n))

    
  #train_df_soil.to_csv('output/train_df_soil.csv')
  #dev_df_soil.to_csv('output/dev_df_soil.csv')
  #test_df_soil.to_csv('output/test_df_soil.csv')
    
    
#intention is for final dataset for training to have sample_size records, distributed over the 5 classes based on the abundance of the class
#  and weighted according to the frequency distribution for the 154 IGSN registrants. Based on 
#  assumption that a given registrant will be documenting similar samples with similar conventions
    
# merge the training dataframes
  theframes = [train_df_min, train_df_rock, train_df_sed,train_df_rocksed,train_df_soil]
  train_df = pd.concat(theframes)
  train_df.sort_values(by='igsn', inplace=True )
  #convert labels into integers
  train_df['iSampleMaterial'] = le.transform(train_df.iSampleMaterial)
  
# merge the dev dataframes
  theframes = [dev_df_min, dev_df_rock, dev_df_sed,dev_df_rocksed,dev_df_soil]
  dev_df = pd.concat(theframes)
  dev_df.sort_values(by='igsn', inplace=True )
  #convert labels into integers
  dev_df['iSampleMaterial'] = le.transform(dev_df.iSampleMaterial)

# merge the training dataframes
  theframes = [test_df_min, test_df_rock, test_df_sed,test_df_rocksed,test_df_soil]
  test_df = pd.concat(theframes) 
  test_df.sort_values(by='igsn', inplace=True )
  #convert labels into integers
  test_df['iSampleMaterial'] = le.transform(test_df.iSampleMaterial)

  train_df.to_csv('output/train_df.csv')
  dev_df.to_csv('output/dev_df.csv')
  test_df.to_csv('output/test_df.csv')
    
  return train_df, dev_df, test_df

In [8]:
## Required parameters
nb_epochs = int(4)  #was 2, then 3, 
batch_size = int(20) #was 10, then 20, tried 30. now 40 w/5000 samples
lr_rate = float(0.007) #was.01

# epochs 4, batch 20, lr_rat .007 worked best yet, with 500 samples; lowest loses at 3 epochs
# try adding axis = 0 in pandas sampling, I can't tell if its using weights. different results-- sed and rocksed are bad,
#  others much better. Run again to see if the same... Get different results. The pandas sample is different, and that 
# impacts the results. 3 of 5 classes identified prttey well. Try raising sample to 1000.  Got matches on 4 classes, 
# good only on mineral, rock, and soil.
# try sampling w/o weights
#  !! worked much better!!

# 2023-08-15
# try 10 epochs (n-1000, .007, batch 20) to see if get any convergence,no convergence, 
#        but recall and precision not bad
# try 3 epochs, rest same. Seemed to work about as well
# try 1 epoch-- seemed to work pretty much the same
# try 3 epoch, 40 batch, 5000 samples-- complete fail! 
# try 3 epoch, 100 batch, 5000 samples
# 1 epoch 100 batch 5000, SS's preprocess. Doesn't get any rockSed, otherwise goot
# 1 epoch 100 batch, 10000 sample, SS preprocess. didn't get any rockSed or soil...
# 4 epoch 100 batch,  10000 sample, SMR preprocess.  TErrible. only got rock
# 4 epoch 100 batch,  10000 sample, ss preprocess.  Terrible. only got rock
# 3 epoch, 20 batch 10000 sample, ss preprocess rate.01 Bust only got rock
# 3 epoch, 20 batch 10000 sample, smr preprocess rate.01 Bust only got rock, but some 
  # convergences
# 4 epoch,batch20,.007,1000 samples, change fractions in training data to favor sed,rocksed. 
    # good matches except rocksed, but some hits theretoo. Got convergence after the first epoch
# try more epochs
# 8 epoch,batch20,.007,1000 samples, change fractions in training data to favor sed,rocksed. 
    # good result rand 42, 23
# results vary slightly with different rand
# raise n to 2000, rand=73, bad result 
# try 4 epoch rand= 73



material_type = str('')

train_mode = str('FALSE')
#train_mode = str('custom')
 #  Whether we account for class imbalance during training by using a custom trainer 
    # (custom) or not (none)                  
output_dir =str('output')
 #Output directory where the model checkpoint will be saved
    

In [9]:
#df = pd.read_csv("iSamplesMaterialTrainingSmall.csv")
# text to read is 'text' column
#df = pd.read_csv("SESARTrainingiSamKeywords.csv", usecols=['igsn', 'traintext'],dtype={'igsn':str,'traintext':str})
# text to read is 'traintext' column
df = pd.read_csv("MaterialTypeData2023-08-07.csv")
# text to read is 'text' column

df = df.fillna("")
    #remove rows that do not have a material type
df = df[df["text"]!=""]

In [10]:
  #load tokenizer 
tokenizer = BertTokenizer.from_pretrained('allenai/scibert_scivocab_uncased', do_lower_case=True, use_fast=True)

#count tokens-- 
####################### SLOW-- this scans all records.
rowcount = 1
ratiosum = 0.0
maxratio = 0.0
for _, row in df.iterrows():
    sentence = row["text"]
    tokens = tokenizer.tokenize(sentence)
    token_count = len(tokens)
    senlen = len(sentence)
    ratio = token_count/senlen
    if ratio > maxratio:
        maxratio = ratio
    
    #print("Original sentence:", sentence)
    #print("Sentence len:", senlen, "; Number of tokens:", token_count, "; ratio:", ratio)
    #print("Number of tokens:", token_count)
    
    rowcount =rowcount + 1
    ratiosum = ratiosum + ratio
    #print("ratio:", ratio)
    
avrage = ratiosum/rowcount
print("Average ratio:", avrage, "; Max ratio:", maxratio)
print("row count: ", rowcount)

    

In [11]:
#load tokenizer
# train(material_type, df, tokenizer, batch_size,lr_rate, nb_epochs, train_mode, output_dir)

# insert train function in line here for debugging...
#train_df, dev_df, test_df = preprocess_2(df)  #steves update
train_df, dev_df, test_df = preprocess(df)  #original function from Sarah Song



 number of labels:  5


In [12]:

#print("train_df columns:", train_df.columns.values)
#print("train_df:", train_df.describe)
#train_df['iSampleMaterial'].values

train_dataset = create_dataset(train_df, tokenizer)
dev_dataset = create_dataset(dev_df,tokenizer)
test_dataset = create_dataset(test_df,tokenizer)

torch tensor dataframe columns: ['igsn' 'isgnprefix' 'text' 'iSampleMaterial' 'weight']
torch tensor dataframe columns: ['igsn' 'isgnprefix' 'text' 'iSampleMaterial' 'weight']
torch tensor dataframe columns: ['igsn' 'isgnprefix' 'text' 'iSampleMaterial' 'weight']


In [13]:
# print(test_dataset.encodings)

In [14]:
# load model
model = BertForSequenceClassification.from_pretrained("allenai/scibert_scivocab_uncased", num_labels = len(le.classes_), )

Some weights of the model checkpoint at allenai/scibert_scivocab_uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification we

In [15]:
desc = model.to(device)
training_args = TrainingArguments(
          no_cuda = True,
          output_dir= output_dir,     # output directory
          num_train_epochs=nb_epochs,              # total number of training epochs
          per_device_train_batch_size=batch_size,  # batch size per device during training
          per_device_eval_batch_size=batch_size,   # batch size for evaluation
          learning_rate = lr_rate,
          warmup_steps=500,                # number of warmup steps for learning rate scheduler
          weight_decay=0.01, 
          load_best_model_at_end=True,            
          logging_dir=output_dir,            # directory for storing logs
          logging_steps=10,
          evaluation_strategy = "epoch", #To calculate metrics per epoch
          save_strategy = "epoch"
  )

In [16]:
  #get class weight
class_weights = get_class_weights(train_df)


In [17]:
CustomTrainer = create_custom_trainer(class_weights)
if train_mode == "custom":
    trainer = CustomTrainer(model = model, args =training_args, train_dataset=train_dataset, eval_dataset=dev_dataset)
else:
    trainer = Trainer(model = model, args =training_args, train_dataset=train_dataset, eval_dataset=dev_dataset)

In [18]:
#train_dataset.labels
#train_dataset.encodings

In [ ]:
trainer.train()

C:\Anaconda\envs\mlclassification\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,1.370100,1.526984
2,1.535400,1.779801


in control.should_save. metrics: {'eval_loss': 1.5269842147827148}
in control.should_save. metrics: {'eval_loss': 1.7798011302947998}


In [ ]:
print(torch.__version__)

In [ ]:
  #conduct evaluation 
  keys = []
  precision = []
  recall = []
  f1 = []

In [ ]:
logits = trainer.predict(test_dataset)[0] #get the logits 

test_pred = np.argmax(logits,axis=-1)
y_test= torch.tensor(test_df['iSampleMaterial'].values.tolist())
res = classification_report(y_test,test_pred,output_dict=True)

In [ ]:
# print(test_dataset.encodings)

In [ ]:
for key, score in res.items():
  if key.isdigit():
    keys.append((le.inverse_transform([int(key)])[0]))
    precision.append(round(score['precision'],2))
    recall.append(round(score['recall'],2))
    f1.append(round(score['f1-score'],2))
    print("%s \t\t\t %0.2f \t %0.2f \t %0.2f"% (le.inverse_transform([int(key)])[0],score['precision'], score['recall'], score['f1-score']))

In [ ]:
#write the results to excel and save
result_df = pd.DataFrame(data=zip(keys,precision,recall,f1), columns=['label','precision','recall','f1'])
result_output_dir ="output/sesar_result.xlsx"
result_df.to_excel(result_output_dir)
print("Macro average: ",f1_score(y_test,test_pred,average='macro'))